In [7]:
from langchain.document_loaders import DirectoryLoader

directory = 'docs'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

/Users/youssefaitalama/Desktop/Portpholio/3155/PocketLawyer/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=500,chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

14828


In [9]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [10]:
query_result = embeddings.embed_query("Hello world")
len(query_result)

384

In [12]:
import pinecone 
from langchain.vectorstores import Pinecone
import os
from dotenv import load_dotenv
load_dotenv()
pinecone.init(api_key=os.getenv('PINECONE_API_KEY'), environment='gcp-starter')

index_name = "lawyer"

index = Pinecone.from_documents(docs, embeddings, index_name=index_name)

In [13]:
def get_similiar_docs(query,k=1,score=False):
  if score:
    similar_docs = index.similarity_search_with_score(query,k=k)
  else:
    similar_docs = index.similarity_search(query,k=k)
  return similar_docs

query = "what is a petty offence"
similar_docs = get_similiar_docs(query)
similar_docs


[Document(page_content='As used in this title, the term ‘‘petty offense’’ means a Class B misdemeanor, a Class C mis- demeanor, or an infraction, for which the maxi- mum fine is no greater than the amount set forth for such an offense in section 3571(b)(6) or (7) in the case of an individual or section 3571(c)(6) or (7) in the case of an organization.\n\n(Added Pub. L. 100–185, § 4(a), Dec. 11, 1987, 101 Stat. 1279; amended Pub. L. 100–690, title VII, § 7089(a), Nov. 18, 1988, 102 Stat. 4409.)', metadata={'source': 'docs/USCODE-2011-title18.pdf'})]